313549 ~ Mateusz Zacharecki \
335719 ~ Patrycja Kielan

---

# Solution for mixed wrapper and filter variable selection
___

# Importing libraries and data

In [6]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import sklearn.model_selection as skm
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
import xgboost
from sklearn.ensemble import VotingRegressor

In [3]:
X_train = pd.read_csv("Xtrain.csv", index_col=0)
X_test = pd.read_csv("Xtest.csv", index_col=0)
y_train = pd.read_csv("ytrain.csv", index_col=0)

# SVM

In [10]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('wrapper', SelectKBest(f_regression, k = 20)),
    ('filter', RFE(DecisionTreeRegressor(), n_features_to_select = 5))
])

## Fitting the whole training set

In [11]:
pipeline_SVM = Pipeline([
    ('pipeline', pipeline),
    ('svm', SVR(C = 10, gamma = 0.1, kernel = 'rbf'))
])

In [12]:
params = {}

grid_pipeline_SVM = GridSearchCV(pipeline_SVM,
                             cv = skm.KFold(3, random_state = 313549, shuffle=True),
                             param_grid = params,
                             scoring = 'neg_mean_squared_error',
                             return_train_score = True)

In [13]:
grid_pipeline_SVM.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=KFold(n_splits=3, random_state=313549, shuffle=True),
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('scaler',
                                                         StandardScaler()),
                                                        ('wrapper',
                                                         SelectKBest(k=20,
                                                                     score_func=<function f_regression at 0x000002752E2E48B0>)),
                                                        ('filter',
                                                         RFE(estimator=DecisionTreeRegressor(),
                                                             n_features_to_select=5))])),
                                       ('svm', SVR(C=10, gamma=0.1))]),
             param_grid={}, return_train_score=True,
             scoring='neg_mean_squared_error')

In [14]:
results_SVM = pd.DataFrame({
                     'mean_test_neg_mean_squared_error': grid_pipeline_SVM.cv_results_["mean_test_score"],
                     'mean_train_neg_mean_squared_error': grid_pipeline_SVM.cv_results_["mean_train_score"],
                     'std_test_neg_mean_squared_error': grid_pipeline_SVM.cv_results_["std_test_score"],
                     'std_train_neg_mean_squared_error': grid_pipeline_SVM.cv_results_["std_train_score"],
                     'rank_test': grid_pipeline_SVM.cv_results_["rank_test_score"]})
results_SVM.sort_values(by = 'rank_test')

,mean_test_neg_mean_squared_error,mean_train_neg_mean_squared_error,std_test_neg_mean_squared_error,std_train_neg_mean_squared_error,rank_test
0,-100.328611,-99.893311,1.112245,0.568079,1


In [15]:
y_pred = pd.DataFrame(grid_pipeline_SVM.predict(X_test))

In [16]:
pd.DataFrame(y_pred).to_csv("SVM_wrapper&filter.txt", sep='\t', index=False, header=False)

# Neural network

In [17]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('wrapper', SelectKBest(f_regression, k = 20)),
    ('filter', RFE(DecisionTreeRegressor(), n_features_to_select = 5))
])

## Fitting the whole training set

In [18]:
pipeline_NN = Pipeline([
    ('pipeline', pipeline),
    ('nn', MLPRegressor(max_iter = 10000, random_state = 313549))
])

In [19]:
params = {
    "nn__hidden_layer_sizes": [(4), (16), (4, 4)],
    "nn__solver": ['sgd', 'adam']
}

In [20]:
grid_pipeline_NN = GridSearchCV(pipeline_NN,
                      cv = skm.KFold(3, random_state = 313549, shuffle=True),
                      param_grid = params,
                      scoring = 'neg_mean_squared_error',
                      return_train_score = True)

In [21]:
grid_pipeline_NN.fit(X_train, y_train.values.ravel())

C:\Users\XXX\anaconda3\lib\site-packages\sklearn\neural_network\_base.py:174: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\XXX\anaconda3\lib\site-packages\sklearn\utils\extmath.py:153: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\XXX\anaconda3\lib\site-packages\sklearn\utils\extmath.py:153: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\XXX\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\XXX\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\XXX\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", 

C:\Users\XXX\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\XXX\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\XXX\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\XXX\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\XXX\anaconda3\lib\site-packages\sklearn\metrics\_regression.py", line 438, in mean_squared_error
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
  File "C:\Users\XXX\anaconda3\lib\site-packages\sklearn\metrics\_regression.py", line 96, in _check_reg_targets
    y_pred = check_a

C:\Users\XXX\anaconda3\lib\site-packages\sklearn\neural_network\_base.py:174: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\XXX\anaconda3\lib\site-packages\sklearn\utils\extmath.py:153: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\XXX\anaconda3\lib\site-packages\sklearn\utils\extmath.py:153: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\XXX\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:131: RuntimeWarning: invalid value encountered in add
  activations[i + 1] += self.intercepts_[i]
C:\Users\XXX\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\XXX\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-tes

GridSearchCV(cv=KFold(n_splits=3, random_state=313549, shuffle=True),
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('scaler',
                                                         StandardScaler()),
                                                        ('wrapper',
                                                         SelectKBest(k=20,
                                                                     score_func=<function f_regression at 0x000002752E2E48B0>)),
                                                        ('filter',
                                                         RFE(estimator=DecisionTreeRegressor(),
                                                             n_features_to_select=5))])),
                                       ('nn',
                                        MLPRegressor(max_iter=10000,
                                                     random_state=313549))]),
             para

In [22]:
results_NN = pd.DataFrame({'nn__hidden_layer_sizes': grid_pipeline_NN.cv_results_["param_nn__hidden_layer_sizes"],
                           'nn__solver': grid_pipeline_NN.cv_results_["param_nn__solver"],
                     'mean_test_rng_mean_squared_error': grid_pipeline_NN.cv_results_["mean_test_score"],
                     'mean_train_neg_mean_squared_error': grid_pipeline_NN.cv_results_["mean_train_score"],
                     'std_test_neg_mean_squared_error': grid_pipeline_NN.cv_results_["std_test_score"],
                     'std_train_neg_mean_squared_error': grid_pipeline_NN.cv_results_["std_train_score"],
                     'rank_test': grid_pipeline_NN.cv_results_["rank_test_score"]})
results_NN.sort_values(by = 'rank_test')

,nn__hidden_layer_sizes,nn__solver,mean_test_rng_mean_squared_error,mean_train_neg_mean_squared_error,std_test_neg_mean_squared_error,std_train_neg_mean_squared_error,rank_test
3,16,adam,-101.985458,-101.598914,1.377431,0.608531,1
1,4,adam,-107.497630,-102.058615,6.527157,1.132712,2
5,"(4, 4)",adam,-107.562755,-103.968276,3.363780,0.497252,3
4,"(4, 4)",sgd,-120.203251,-120.202642,1.288286,0.643973,4
0,4,sgd,NaN,NaN,NaN,NaN,5
2,16,sgd,NaN,NaN,NaN,NaN,6


In [23]:
y_pred = pd.DataFrame(grid_pipeline_NN.predict(X_test))

In [24]:
pd.DataFrame(y_pred).to_csv("neural_network_wrapper&filter.txt", sep='\t', index=False, header=False)

# XGBoost

In [26]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('wrapper', SelectKBest(f_regression, k = 20)),
    ('filter', RFE(DecisionTreeRegressor(), n_features_to_select = 5))
])

## Fitting the whole training set

In [28]:
pipeline_xgb = Pipeline([
    ('pipeline', pipeline),
    ('xgb', xgboost.XGBRegressor(objective='reg:squarederror', sampling_method = "hist", n_estimators = 400, random_state = 313549))
])

In [29]:
params = {
    "xgb__eta": [0.3, 0.1, 1],
    "xgb__gamma": [0, 0.1, 1],
    "xgb__max_depth": [10, 20]
}

In [30]:
grid_pipeline_xgb = GridSearchCV(pipeline_xgb,
                      cv = skm.KFold(3, random_state = 313549, shuffle=True),
                      param_grid = params,
                      scoring = 'neg_mean_squared_error',
                      return_train_score = True)

In [31]:
grid_pipeline_xgb.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=KFold(n_splits=3, random_state=313549, shuffle=True),
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('scaler',
                                                         StandardScaler()),
                                                        ('wrapper',
                                                         SelectKBest(k=20,
                                                                     score_func=<function f_regression at 0x000002752E2E48B0>)),
                                                        ('filter',
                                                         RFE(estimator=DecisionTreeRegressor(),
                                                             n_features_to_select=5))])),
                                       ('xgb',
                                        XGBRegressor(base_score=None,
                                                     boost...
                               

In [43]:
results_xgb = pd.DataFrame({'xgb__eta': grid_pipeline_xgb.cv_results_["param_xgb__eta"],
                            'xgb__gamma': grid_pipeline_xgb.cv_results_["param_xgb__gamma"],
                            'xgb__max_depth': grid_pipeline_xgb.cv_results_["param_xgb__max_depth"],
                     'mean_test_neg_mean_squared_error': grid_pipeline_xgb.cv_results_["mean_test_score"],
                     'mean_train_neg_mean_squared_error': grid_pipeline_xgb.cv_results_["mean_train_score"],
                     'std_test_neg_mean_squared_error': grid_pipeline_xgb.cv_results_["std_test_score"],
                     'std_train_neg_mean_squared_error': grid_pipeline_xgb.cv_results_["std_train_score"],
                     'rank_test': grid_pipeline_xgb.cv_results_["rank_test_score"]})
results_xgb.sort_values(by = 'rank_test')

,xgb__eta,xgb__gamma,xgb__max_depth,mean_test_neg_mean_squared_error,mean_train_neg_mean_squared_error,std_test_neg_mean_squared_error,std_train_neg_mean_squared_error,rank_test
6,0.1,0,10,-97.472681,-6.324241e+01,0.894526,9.616894e-01,1
8,0.1,0.1,10,-97.571059,-6.244620e+01,0.983733,8.379006e-01,2
10,0.1,1,10,-97.624602,-6.314651e+01,0.885826,3.245240e-01,3
9,0.1,0.1,20,-101.816029,-1.441798e+01,0.945458,1.335207e-02,4
7,0.1,0,20,-101.939872,-1.438424e+01,1.120746,7.946069e-01,5
11,0.1,1,20,-102.322042,-1.393394e+01,0.602373,6.813998e-01,6
0,0.3,0,10,-105.703395,-3.334883e+01,0.824592,4.964421e-01,7
4,0.3,1,10,-105.822980,-3.321209e+01,0.475823,3.129403e-01,8
2,0.3,0.1,10,-105.907167,-3.358913e+01,0.568425,5.477816e-01,9
3,0.3,0.1,20,-112.373963,-3.474772e-01,0.898167,1.338751e-02,10


In [33]:
y_pred = pd.DataFrame(grid_pipeline_xgb.predict(X_test))

In [34]:
pd.DataFrame(y_pred).to_csv("xgboost_wrapper&filter.txt", sep='\t', index=False, header=False)

# Linear regression

In [35]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('wrapper', SelectKBest(f_regression, k = 20)),
    ('filter', RFE(DecisionTreeRegressor(), n_features_to_select = 5))
])

## Fitting the whole training set

In [36]:
pipeline_lm = Pipeline([
    ('pipeline', pipeline),
    ('lm', LinearRegression())
])

In [37]:
params = {}

In [38]:
grid_pipeline_lm = GridSearchCV(pipeline_lm,
                      cv = skm.KFold(3, random_state = 313549, shuffle=True),
                      param_grid = params,
                      scoring = 'neg_mean_squared_error',
                      return_train_score = True)

In [39]:
grid_pipeline_lm.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=KFold(n_splits=3, random_state=313549, shuffle=True),
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('scaler',
                                                         StandardScaler()),
                                                        ('wrapper',
                                                         SelectKBest(k=20,
                                                                     score_func=<function f_regression at 0x000002752E2E48B0>)),
                                                        ('filter',
                                                         RFE(estimator=DecisionTreeRegressor(),
                                                             n_features_to_select=5))])),
                                       ('lm', LinearRegression())]),
             param_grid={}, return_train_score=True,
             scoring='neg_mean_squared_error')

In [40]:
results_lm = pd.DataFrame({
                     'mean_test_neg_mean_squared_error': grid_pipeline_lm.cv_results_["mean_test_score"],
                     'mean_train_neg_mean_squared_error': grid_pipeline_lm.cv_results_["mean_train_score"],
                     'std_test_neg_mean_squared_error': grid_pipeline_lm.cv_results_["std_test_score"],
                     'std_train_neg_mean_squared_error': grid_pipeline_lm.cv_results_["std_train_score"],
                     'rank_test': grid_pipeline_lm.cv_results_["rank_test_score"]})
results_lm.sort_values(by = 'rank_test')

,mean_test_neg_mean_squared_error,mean_train_neg_mean_squared_error,std_test_neg_mean_squared_error,std_train_neg_mean_squared_error,rank_test
0,-102.898103,-102.889776,1.033197,0.516483,1


In [41]:
y_pred = pd.DataFrame(grid_pipeline_lm.predict(X_test))

In [42]:
pd.DataFrame(y_pred).to_csv("linear_regression_wrapper&filter.txt", sep='\t', index=False, header=False)

# Voting

In [4]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('wrapper', SelectKBest(f_regression, k = 20)),
    ('filter', RFE(DecisionTreeRegressor(), n_features_to_select = 5))
])

## Fitting the whole data set

In [7]:
reg1 = SVR(C = 10, gamma = 0.1, kernel = 'rbf')
reg2 = MLPRegressor(hidden_layer_sizes = 16, solver = 'adam', max_iter = 10000, random_state = 313549)
reg3 = xgboost.XGBRegressor(eta = 0.1, gamma = 0, max_depth = 10, objective='reg:squarederror', sampling_method = "hist", n_estimators = 400, random_state = 313549)

pipeline_voting = Pipeline([
    ('pipeline', pipeline),
    ('eclf', VotingRegressor(estimators=[('svm', reg1),
                             ('nn', reg2),
                             ('xgb', reg3)]))
])

In [8]:
params = {}

grid_pipeline_voting = GridSearchCV(pipeline_voting,
                      cv = skm.KFold(3, random_state = 313549, shuffle=True),
                      param_grid = params,
                      scoring = 'neg_mean_squared_error',
                      return_train_score = True)

In [9]:
grid_pipeline_voting.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=KFold(n_splits=3, random_state=313549, shuffle=True),
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('scaler',
                                                         StandardScaler()),
                                                        ('wrapper',
                                                         SelectKBest(k=20,
                                                                     score_func=<function f_regression at 0x000002B54C0268B0>)),
                                                        ('filter',
                                                         RFE(estimator=DecisionTreeRegressor(),
                                                             n_features_to_select=5))])),
                                       ('eclf',
                                        VotingRegressor(estimators=[('svm...
                                                                                  min

In [10]:
results_voting = pd.DataFrame({
                     'mean_test_neg_mean_squared_error': grid_pipeline_voting.cv_results_["mean_test_score"],
                     'mean_train_neg_mean_squared_error': grid_pipeline_voting.cv_results_["mean_train_score"],
                     'std_test_neg_mean_squared_error': grid_pipeline_voting.cv_results_["std_test_score"],
                     'std_train_neg_mean_squared_error': grid_pipeline_voting.cv_results_["std_train_score"],
                     'rank_test': grid_pipeline_voting.cv_results_["rank_test_score"]})
results_voting.sort_values(by = 'rank_test')

,mean_test_neg_mean_squared_error,mean_train_neg_mean_squared_error,std_test_neg_mean_squared_error,std_train_neg_mean_squared_error,rank_test
0,-96.295714,-83.52281,1.057334,0.735309,1


In [11]:
y_pred = pd.DataFrame(grid_pipeline_voting.predict(X_test))

In [12]:
pd.DataFrame(y_pred).to_csv("voting_wrapper&filter.txt", sep='\t', index=False, header=False)